In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.util.remote import Remote
from pymoo.problems import get_problem
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.operators.sampling.rnd import FloatRandomSampling,IntegerRandomSampling
from pymoo.operators.repair.rounding import RoundingRepair
import time
import matplotlib.pyplot as plt
import math
import copy

## 1ª ETAPA

In [2]:
## Função para calcular o retorno logarítmico
def log_return(series):
    return np.log(series / series.shift(1))

In [3]:
# Criptomoedas que vamos escolher para nosso portifólio
cryptos = [
    'BTC-USD', 'ETH-USD','LTC-USD', 'ADA-USD',
    'DOT-USD', 'LINK-USD','SOL-USD',
    'TRX-USD'
]

* Agora vamos criar um dicionário chamado data
* Cada elemento de data é uma criptomoeda
* data[crypto_X] representa uma tabela, de 1513 rows × 1 columns, excluindo a coluna de index.
* A primeira coluna é o dia e a segunda coluna é o **retorno esperado que aquele dia deveria ter previsto**

In [4]:
data_complete = {}
data = {}
for crypto in cryptos:
    # Baixar dados e calcular retorno logarítmico
    df = yf.download(crypto, start='2020-08-21', end=pd.to_datetime("today").strftime("%Y-%m-%d"))
    df['Return'] = log_return(df['Close']).shift(-2)

    # Construir DataFrame final, pegando o retorno do dia anterior e o atual
    df_final = pd.DataFrame({
        'Crypto_Return_Day_1': df['Return'].shift(1),
        'Crypto_Return_Today': df['Return']
    }).dropna()

    data[crypto] = df_final[['Crypto_Return_Today']]

data_complete = copy.deepcopy(data)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
data_complete = {}
for crypto in cryptos:
    # Baixar dados e calcular retorno logarítmico
    df = yf.download(crypto, start='2020-08-21', end=pd.to_datetime("today").strftime("%Y-%m-%d"))
    df['Return'] = log_return(df['Close']).shift(-2)

    # Construir DataFrame final, pegando o retorno do dia anterior e o atual
    df_final = pd.DataFrame({
        'Crypto_Return_Day_1': df['Return'].shift(1),
        'Crypto_Return_Today': df['Return']
    }).dropna()

    # Adicionar a coluna 'exp_value' com valores NaN
    df_final['exp_value'] = np.nan

    # Armazenar no dicionário
    data_complete[crypto] = df_final[['Crypto_Return_Today', 'exp_value']]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [6]:
data['ETH-USD']
data_complete['ETH-USD']

,Crypto_Return_Today,exp_value
Date,,
2020-08-22,0.041930,NaN
2020-08-23,-0.060975,NaN
2020-08-24,0.006399,NaN
2020-08-25,-0.009969,NaN
2020-08-26,0.034022,NaN
...,...,...
2024-10-08,0.006555,NaN
2024-10-09,0.021848,NaN
2024-10-10,0.016288,NaN


#### Realizei uma pequena adição
#### Agora em vez de apenas data, temos também data_complete, que será um dicionário que não vamos alterar, sempre contendo apenas uma tabela com todos os dias e os retornos que deveriam ser previstos naquele dia

# 2ª ETAPA


* Transformaremos cada elemento do dicionário data[crypto] em um vetor de tabelas
* desse modo, data[crypto] agora é um vetor com 1212 elementos, cada elemento sendo uma tabela de W+1 linhas e uma coluna

In [7]:
W = 300      ####     HIPER PARAMETRO     #####

# Criando a função First Transformation, que recebe uma tabela contendo os retornos de todos os dias dos ultimos X anos e retorno um vetor, que cada elemento
# é uma tabela do retorno de W+1 dias.
def FirstTransform(df, W):
    vet = []
    Linhas, Colunas = df.shape
    for i in range(Linhas, W+1, -1):
        vet.append(df.iloc[(i-W-1):i])
    return vet
for crypto in cryptos:
    data[crypto] = FirstTransform(data[crypto], W)

In [8]:
print(len(data['ETH-USD']))
data['ETH-USD'][0]

1212


,Crypto_Return_Today
Date,
2023-12-17,-0.017930
2023-12-18,0.010977
2023-12-19,0.016946
2023-12-20,0.038104
2023-12-21,-0.007541
...,...
2024-10-08,0.006555
2024-10-09,0.021848
2024-10-10,0.016288


In [9]:
# Definindo uma função bagging, que recebe um dataframe e retorna um vetor de dataframes.
def Bagging(df, n, gamma):
    df_bagged = []
    for i in range(gamma):
        aux = df.sample(n = n, random_state = i)
        aux = aux.sort_index()
        df_bagged.append(aux)
    return df_bagged

### Por motivos de simplicidade, primeiros vamos fazer todo o programa sem bagging e sem purge K-fold Cross Validation

In [10]:
test_target = {}
for crypto in cryptos:
    row = []
    for i in range(len(data[crypto])):
        row.append((pd.DataFrame(data[crypto][i].iloc[-1])).T)
        data[crypto][i] = data[crypto][i].drop(data[crypto][i].index[-1])
    test_target[crypto] = row

In [11]:
data['ETH-USD'][0]
test_target['ETH-USD'][0]

,Crypto_Return_Today
2024-10-12,0.063288


# 3ª ETAPA

* Model(df):    FUNÇÃO QUE RECEBE UM DATAFRAME E RETORNA O VALOR ESPERADO DO ULTIMO ELEMENTO DESSE DATAFRAME
* REALIZAMOS O TREINO COM TODO MUNDO, MENOS COM ESSE ULTIMO ELEMENTO AI

y_lstm é um np.array de shape (296,1)

X_lstm é um np.array de shape (296, timesteps, 1)

In [12]:
def Model(df):
    # Normalizando o dataframe
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df)
    df_array = np.array(df_scaled)

    # Transformando no formato lstm
    X_lstm =[]
    y_lstm = []
    timesteps = 15
    for i in range(timesteps, len(df_array)):
        X_lstm.append(df_array[i-timesteps:i, :])  # Pega 'timesteps' linhas anteriores
        y_lstm.append(df_scaled[i])  # Alvo é o valor do dia seguinte
    X_lstm = np.array(X_lstm)
    y_lstm = np.array(y_lstm)

    
    # Separar o X_train (todos exceto o último elemento)
    X_train = X_lstm[:-1]
    y_train = y_lstm[:-1]

    # X_test e y_test
    X_test = X_lstm[-1:]
    y_test = y_lstm[-1:]

    # Defina o modelo
    model = Sequential()
    model.add(LSTM(units=15, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2]))) # [samples, time steps, features]
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Treine o modelo
    early_stopping = EarlyStopping(monitor='loss', patience=3)  # Para o treinamento quando a perda parar de melhorar
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose = 0, callbacks=[early_stopping])

    y_hat_scaled = model.predict(X_test)
    y_hat = scaler.inverse_transform(y_hat_scaled)[0][0]

    return y_hat

#### IDEIA: ADICIONAR UMA COLUNA AO MEU DICIONARIO DATA_COMPLETE, EM QUE ESSA COLUNA CORRESPONDE AO VALOR PREVISTO PELO MODELO PARA AQUELE DIA.

In [13]:
data_complete

{'BTC-USD':             Crypto_Return_Today  exp_value
 Date                                      
 2020-08-22             0.009364        NaN
 2020-08-23            -0.035306        NaN
 2020-08-24             0.010696        NaN
 2020-08-25            -0.014463        NaN
 2020-08-26             0.019165        NaN
 ...                         ...        ...
 2024-10-08            -0.005090        NaN
 2024-10-09             0.035378        NaN
 2024-10-10             0.011906        NaN
 2024-10-11            -0.005421        NaN
 2024-10-12             0.049581        NaN
 
 [1513 rows x 2 columns],
 'ETH-USD':             Crypto_Return_Today  exp_value
 Date                                      
 2020-08-22             0.041930        NaN
 2020-08-23            -0.060975        NaN
 2020-08-24             0.006399        NaN
 2020-08-25            -0.009969        NaN
 2020-08-26             0.034022        NaN
 ...                         ...        ...
 2024-10-08             0.

In [14]:
def Add(y_hat, data_complete, ind, crypto):
    data_complete[crypto].at[ind, 'exp_value'] = y_hat


In [15]:
len(data['ETH-USD'])

1212

In [16]:
D = 60 # Dias que vamos testar nosso modelo.
# for day in days:
for day in range(0, D):
    for crypto in cryptos:
        df_train = data[crypto][day].copy() # df é uma tabela de 300 linhas e uma coluna, que consiste no conjunto de treinamento para o dia de hoje
        df_test = test_target[crypto][day].copy() # df_test é uma tabela com 1 linha e uma coluna, que consite na resposta do meu dia atual

        df = pd.concat([df_train, df_test], ignore_index=True) # unindo as duas tabelas em um df apenas, para enviar para minha função modelo

        y_hat = Model(df)

        # quero adicionar esse y_hat na coluna de exp_return na linha do meu dia i, no meu data_complete[crypto]
        Add(y_hat, data_complete, df_test.index[0], crypto)

C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step


C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step


C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step


C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step


In [18]:
for crypto in cryptos:
    # Define o nome do arquivo como o nome da criptomoeda seguido de .csv
    filename = f"{crypto}.csv"
    
    # Salva o DataFrame data_complete[crypto] no arquivo .csv
    data_complete[crypto].to_csv(filename, index=False)

    print(f"Arquivo {filename} salvo com sucesso!")

Arquivo BTC-USD.csv salvo com sucesso!
Arquivo ETH-USD.csv salvo com sucesso!
Arquivo LTC-USD.csv salvo com sucesso!
Arquivo ADA-USD.csv salvo com sucesso!
Arquivo DOT-USD.csv salvo com sucesso!
Arquivo LINK-USD.csv salvo com sucesso!
Arquivo SOL-USD.csv salvo com sucesso!
Arquivo TRX-USD.csv salvo com sucesso!


In [19]:
data_complete['ETH-USD']

,Crypto_Return_Today,exp_value
Date,,
2020-08-22,0.041930,NaN
2020-08-23,-0.060975,NaN
2020-08-24,0.006399,NaN
2020-08-25,-0.009969,NaN
2020-08-26,0.034022,NaN
...,...,...
2024-10-08,0.006555,-0.005546
2024-10-09,0.021848,-0.002075
2024-10-10,0.016288,-0.000652


In [24]:
for crypto in cryptos:
    # Calcular a diferença entre Crypto_Return_Today e exp_value
    # Calcular o desvio padrão do Crypto_Return_Today nos últimos 30 dias
    data_complete[crypto]['volatility'] = data_complete[crypto]['Crypto_Return_Today'].rolling(window=30).std()


In [25]:
data_complete['ETH-USD']

,Crypto_Return_Today,exp_value,volatilidade,volatility
Date,,,,
2020-08-22,0.041930,NaN,NaN,NaN
2020-08-23,-0.060975,NaN,NaN,NaN
2020-08-24,0.006399,NaN,NaN,NaN
2020-08-25,-0.009969,NaN,NaN,NaN
2020-08-26,0.034022,NaN,NaN,NaN
...,...,...,...,...
2024-10-08,0.006555,-0.005546,0.025827,0.024612
2024-10-09,0.021848,-0.002075,0.025928,0.024598
2024-10-10,0.016288,-0.000652,0.026061,0.024709


In [16]:

df_train = data['ETH-USD'][0].copy() # df é uma tabela de 300 linhas e uma coluna, que consiste no conjunto de treinamento para o dia de hoje
df_test = test_target['ETH-USD'][0].copy() # df_test é uma tabela com 1 linha e uma coluna, que consite na resposta do meu dia atual

df = pd.concat([df_train, df_test], ignore_index=False)
y_hat  = 16

Add(y_hat, data_complete, df_test.index[0], 'ETH-USD')

In [17]:
df

scaler = MinMaxScaler()

df_scaled = scaler.fit_transform(df)

pd.DataFrame(df_scaled)

df_array = np.array(df_scaled)

X_lstm =[]
y_lstm = []
timesteps = 5
for i in range(timesteps, len(df_array)):
    X_lstm.append(df_array[i-timesteps:i, :])  # Pega 'timesteps' linhas anteriores
    y_lstm.append(df_scaled[i])  # Alvo é o valor do dia seguinte
X_lstm = np.array(X_lstm)
y_lstm = np.array(y_lstm)



# Separar o X_train (todos exceto o último elemento)
X_train = X_lstm[:-1]
y_train = y_lstm[:-1]

# X_test e y_test
X_test = X_lstm[-1:]
y_test = y_lstm[-1:]



In [49]:
day = 1
for crypto in cryptos:
    df_train = data[crypto][day].copy() # df é uma tabela de 300 linhas e uma coluna, que consiste no conjunto de treinamento para o dia de hoje
    df_test = test_target[crypto][day].copy() # df_test é uma tabela com 1 linha e uma coluna, que consite na resposta do meu dia atual

    df = pd.concat([df_train, df_test], ignore_index=True) # unindo as duas tabelas em um df apenas, para enviar para minha função modelo

    y_hat = Model(df)

    # quero adicionar esse y_hat na coluna de exp_return na linha do meu dia i, no meu data_complete[crypto]
    Add(y_hat, data_complete, df_test.index[0], crypto)

C:\Users\delve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


TypeError: InteractiveShell.showtraceback.<locals>.contains_exceptiongroup() takes 1 positional argument but 2 were given

In [23]:
# 16s por dia
data_complete['ETH-USD']

,Crypto_Return_Today,exp_value
Date,,
2020-08-22,0.041930,NaN
2020-08-23,-0.060975,NaN
2020-08-24,0.006399,NaN
2020-08-25,-0.009969,NaN
2020-08-26,0.034022,NaN
...,...,...
2024-10-08,0.006555,NaN
2024-10-09,0.021848,NaN
2024-10-10,0.016288,NaN
